methodology [here](https://hackmd.io/twOh6VgqSnyaAy8BbYrVLg?edit)

## installation and set up

In [ ]:
!pip install openai

from IPython.display import clear_output

clear_output()

from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from openai import OpenAI
import json

client = OpenAI(
    api_key = OPENAI_API_KEY
)

client = OpenAI(api_key = OPENAI_API_KEY)

main_instructions = """
You are a helpful assistant that will be deployed for multiple different functions focused on generating narratives based on the Little Red Riding Hood tale.
You have access to a number of files, and you should retrieve passages from them to help guide your activity.
You also have access to a number of tools (functions to call), and it is important that you choose the right functions to use.
"""

assistant = client.beta.assistants.create(
    name="demo-bot",
    instructions=main_instructions,
    tools=[{"type": "retrieval"}],
    model="gpt-4-turbo-preview"
)

print(json.dumps(assistant.dict(), indent=4))


In [ ]:
# upload files and store as `files`
import os

# Path to your assistant documents
assistant_docs_path = "/content/drive/MyDrive"

assistant_id=assistant.id

# Initialize an empty list to hold the file IDs
uploaded_file_ids = []

# Iterate over each file in the directory
for filename in os.listdir(assistant_docs_path):
    print(filename)
    file_path = os.path.join(assistant_docs_path, filename)
    print(file_path)

    # Check if it's a file (and not a directory)
    if os.path.isfile(file_path):
        print(f"Uploading {filename}...")
        # Upload the file and store the response
        response = client.files.create(
            file=open(file_path, "rb"),
            purpose="assistants"
        )

        print(response.dict())
        uploaded_file_ids.append(response.id)
        print(f"Uploaded {filename} and received id: {response.id}")

# Now, uploaded_file_ids contains all the uploaded file IDs
print("Uploaded file IDs:", uploaded_file_ids)




In [ ]:
print(uploaded_file_ids)

In [ ]:
print(assistant.id)

In [ ]:
# update assistant with retrieval

import datetime
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# from previous step in this colab
new_assisant_name = f"demo-bot-w-retrieval-{timestamp}"


my_updated_assistant = client.beta.assistants.update(
  assistant.id,
  name=new_assisant_name,
  tools=[{"type": "retrieval"}],
  model="gpt-4-turbo-preview",
  file_ids=uploaded_file_ids,
)

print(my_updated_assistant)